In [1]:
import boto3, json, math

class ToolError(Exception):
    pass


README.md		  complaints_tooluse_cleaned.ipynb
complaints_tooluse.ipynb  tooluse_multiple_example-Copy1.ipynb


In [2]:
import boto3, json, math
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain.embeddings import BedrockEmbeddings
import pandas as pd

# Initialize the Bedrock runtime client
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name='us-east-1',
)

# Create an instance of BedrockEmbeddings using the Bedrock client
embeddings = BedrockEmbeddings(
    client=bedrock_client, 
    model_id="amazon.titan-embed-text-v2:0"
)
try:
# Load the previously created FAISS vector store for complaints
    vector_store = FAISS.load_local('complaints.vs', embeddings, allow_dangerous_deserialization=True)
except:
    import faiss
    from langchain_community.docstore.in_memory import InMemoryDocstore
    from langchain_community.vectorstores import FAISS
    from uuid import uuid4
    from langchain_core.documents import Document
    documents = []
    df = pd.read_csv(r'complaints_fake.csv')
    for index, row in df.iterrows():
        document_temp = Document(page_content=row['complaint_text'],metadata={"client_name": row['client_name'],"client_region": row['client_region'],"theme": row['theme'],"complaint_date": row['complaint_date']})
        documents.append(document_temp)
    vector_store = FAISS.from_documents(documents=documents,embedding=embeddings)


/tmp/ipykernel_5173/706041567.py:15: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import BedrockEmbeddings`.
  embeddings = BedrockEmbeddings(


FileNotFoundError: [Errno 2] No such file or directory: 'complaints_fake.csv'

In [ ]:

def getContext(user_prompt:str,filter_terms:dict):
    """
    This function retrieves the context (complaints) based on the user's query.

    Parameters:
    user_prompt (str): The user's query.
    vector_store: The FAISS vector store containing the complaints.
    tool_list (list): The list of tools to use for filtering.

    Returns:
    pd.DataFrame: A DataFrame containing the retrieved complaints.
    """
    try:
        # Extract the metadata filter and the number of documents to return from the filter response
        metadata_filter = filter_terms['metadata_filter']
        k_filter = filter_terms['k_filter']
        print(f"filter by metadata: {metadata_filter} and k: {k_filter}")
        
        # Create a retriever with the extracted filters
        retriever = vector_store.as_retriever(search_kwargs={'filter': metadata_filter, 'k':k_filter})
    except:
        print('cannot extract out filters so going default')
        # Create a retriever with default settings if filters cannot be extracted
        retriever = vector_store.as_retriever(search_kwargs= {'k':100})

    # Retrieve the documents using the retriever
    docs = retriever.invoke(user_prompt)
    
    # Create a DataFrame to store the retrieved complaints
    master_df = pd.DataFrame()
    for doc in docs:
        test=doc.metadata
        test.update({'complaint_text':doc.page_content})
        temp_df = pd.DataFrame([test.values()],columns=test.keys())
        master_df = pd.concat([master_df,temp_df])
    
    return master_df

In [ ]:
import boto3
import json

# Initialize the Bedrock runtime client
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name='us-east-1',
)

def call_bedrock(message_list, system_prompts, tool_list):
    """
    This function interacts with Amazon Bedrock Converse API to generate responses using a specified model and tools.

    Parameters:
    message_list (list): A list of messages to send to the model.
    system_prompts (str): System prompts to guide the model's response.
    tool_list (list): A list of tools to be used by the model.

    Returns:
    dict: The response from the Bedrock service.
    """
    
    # Call the Bedrock converse API with tool configuration
    response = bedrock_client.converse(
        modelId="amazon.nova-pro-v1:0",  # ID of the model to use
        messages=message_list,  # Messages to send to the model
        system=[{ 'text': system_prompts }],  # System prompts
        inferenceConfig={  # Inference configuration
            "maxTokens": 2000,  # Maximum number of tokens to generate
            "temperature": 0.1  # Temperature for response randomness
        },
        toolConfig={ "tools": tool_list }  # Tool configuration
    )
    
    # Return the response from the Bedrock service
    return response

def process_tool_uses(message_list, system_prompts, tool_list):
    """
    This function processes tool uses iteratively until a final response is generated.

    Parameters:
    message_list (list): A list of messages to send to the model.
    system_prompts (str): System prompts to guide the model's response.
    tool_list (list): A list of tools to be used by the model.

    Returns:
    str: The final generated response.
    """
    
    while True:
        response = call_bedrock(message_list, system_prompts, tool_list)
        print(response)
        for content in response['output']['message']['content']:
            if 'toolUse' in content:
                tool_use = content['toolUse']
                
                tool_name = tool_use['name']
                tool_input = tool_use['input']
                
                if tool_name == "identify_complaints_filters":
                    # Simulate identifying filters based on the query
                    filters = tool_input
                    print(f"--------------------------------\n Identified filters: {filters}")
                    
                    # Update the tool list to call the next tool with the identified filters
                    tool_list = [
                        {
                            "toolSpec": {
                                "name": "get_complaints_data",
                                "input": {
                                    "filters": filters
                                }
                            }
                        }
                    ]
                    break  # Break to re-call the API with the updated tool list
                
            
            else:
                # Extract the final response if no tool uses are present
                final_response = response['output']['message']['content'][0]['text']
                print(f"Generated response: {final_response}")
                return final_response

# Define the tool list
tool_list =[
    {
        "toolSpec": {
            "name": "identify_complaints_filters",
            "description": """"Your job is to first look at the query's query and determine if it requires filtering the complaints first based off the following columns 
            client_name - name of the company making the complaint, you will need to use the like filter since users might not type if the exact name
            client_region - possible values ['MC','LC','ICB'] where MC= mid-corporate, LC = Large corporate, ICB = International corprorate
            complaint_date - date of the complaint

            Note: if you're not sure then don't output anything. This is used for metadata filter for the vectorstore. There are two outputs x and y, where x is the column filters and y is the number of documents to return - if user doesnt specify then default is 100

            Example1: user_query: Show me all complaints in region MC
            output: = {'metadata_filter': {client_region": "MC"},'k_filter':100}

            Example2: user_query: Show me all complaints in region LC and after 15th June 2024
            output: {'metadata_filter':{"client_region": "MC","complaint_date": ">2024-06-15"},'k_filter':100}


            Example3: user_query: Show me 5 complaints in regions MC and LC
            output: {'metadata_filter':{client_region": ["MC",'LC']},'k_filter':5}  """,
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "filter_dict": {
                            "type": "dict",
                            "description": """output filter as a dict e.g. {'metadata_filter':{client_region": ["MC",'LC']},'k_filter':5} """
                        }
                    },
                    "required": ["filter_dict"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "get_complaintsData",
            "description": """This function retrieves the context (complaints) based on the user's query.""",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "user_query": {
                            "type": "str",
                            "description": """original user query"""
                        },
                         "filter_terms": {
                            "type": "dict",
                            "description": """filter term from the output of tool identify_complaints_filters"""
                        }
                    },
                    "required": ["user_query","filter_terms"]
                }
            }
        }
    }
]

# Define the message list and system prompts
message_list = [
    {
        "role": "user",
        "content": [ { "text": "Show me all complaints in region MC after 15th June 2024" } ]
    }
]

system_prompts = """You are an AI assistant within a corporate bank in the Complaints team. 
Your role is to retrieve and analyze complaints based on user queries. 
Use the provided tools to filter, retrieve, and generate responses."""

# # Process tool uses iteratively
# final_response = process_tool_uses(message_list, system_prompts, tool_list)
# print(f"Final response: {final_response}")

In [ ]:
def get_tool_result(tool_use_block):
    
    tool_use_name = tool_use_block['name']
            
    print(f"Using tool {tool_use_name}")
    
    # Note: We're deliberately excluding tangent so something magical can happen
    if tool_use_name == 'identify_complaints_filters':
        return math.cos(tool_use_block['input']['x'])
    elif tool_use_name == 'sine':
        return math.sin(tool_use_block['input']['x'])
    elif tool_use_name == 'divide_numbers':
        return tool_use_block['input']['x'] / tool_use_block['input']['y'] 
    else:
        raise ToolError(f"Invalid function name: {tool_use_name}")

In [ ]:
# response = call_bedrock(message_list, system_prompts, tool_list)

In [ ]:
# response

In [ ]:
for content in response['output']['message']['content']:
    if 'toolUse' in content:
        tool_use = content['toolUse']
        
        tool_name = tool_use['name']
        tool_input = tool_use['input']
        
        if tool_use['name']=='identify_complaints_filters':
            try:
                    follow_up_content_blocks.append({
                        "toolResult": {
                            "toolUseId": tool_use_block['toolUseId'],
                            "toolUseName": "identify_complaints_filters",
                            "content": [
                                { "json": { "filter_terms": tool_use_block['input'] } }
                            ]
                        }
                    })
                
            except ToolError as e:
                follow_up_content_blocks.append({ 
                    "toolResult": {
                        "toolUseId": tool_use_block['toolUseId'],
                        "toolUseName": "identify_complaints_filters",
                        "content": [  { "text": repr(e) } ],
                        "status": "error"
                    }
                })
    
    

In [ ]:
def get_tool_result(tool_use_block):
    
    tool_use_name = tool_use_block['name']
            
    print(f"Using tool {tool_use_name}")
    
    # Note: We're deliberately excluding tangent so something magical can happen
    if tool_use_name == 'cosine':
        return math.cos(tool_use_block['input']['x'])
    elif tool_use_name == 'sine':
        return math.sin(tool_use_block['input']['x'])
    elif tool_use_name == 'divide_numbers':
        return tool_use_block['input']['x'] / tool_use_block['input']['y'] 
    else:
        raise ToolError(f"Invalid function name: {tool_use_name}")


In [ ]:
def handle_response(response_message):
    
    response_content_blocks = response_message['content']
    
    follow_up_content_blocks = []
    
    for content_block in response_content_blocks:
        if 'toolUse' in content_block:
            tool_use_block = content_block['toolUse']

            if tool_use_block['name']=='identify_complaints_filters':
                try:
                        follow_up_content_blocks.append({
                            "toolResult": {
                                "toolUseId": tool_use_block['toolUseId'],
                                "content": [
                                    { "json": { "filter_terms": tool_use_block['input'] } }
                                ]
                            }
                        })
                    
                except ToolError as e:
                    follow_up_content_blocks.append({ 
                        "toolResult": {
                            "toolUseId": tool_use_block['toolUseId'],
                            "content": [  { "text": repr(e) } ],
                            "status": "error"
                        }
                    })
            if tool_use_block['name']=='get_complaintsData':
                user_query =  tool_use_block['input']['user_query']
                filter_terms = tool_use_block['input']['filter_terms']
                
    
    if len(follow_up_content_blocks) > 0:
        
        follow_up_message = {
            "role": "user",
            "content": follow_up_content_blocks,
        }
        
        return follow_up_message
    else:
        return None


In [ ]:
def run_loop(prompt,system_prompts, tool_list):
    MAX_LOOPS = 6
    loop_count = 0
    continue_loop = True
    
    message_list = [
        {
            "role": "user",
            "content": [ { "text": prompt } ]
        }
    ]
    
    while continue_loop:
        response = call_bedrock(message_list,system_prompts, tool_list)
        
        response_message = response['output']['message']
        message_list.append(response_message)
        
        loop_count = loop_count + 1
        
        if loop_count >= MAX_LOOPS:
            print(f"Hit loop limit: {loop_count}")
            break
        
        follow_up_message = handle_response(response_message)
        
        if follow_up_message is None:
            # No remaining work to do, return final response to user
            continue_loop = False 
        else:
            message_list.append(follow_up_message)
            
    return message_list


In [ ]:
message_list = [
    {
        "role": "user",
        "content": [ { "text": "Show me all complaints in region MC after 15th June 2024" } ]
    }
]

system_prompts = """You are an AI assistant within a corporate bank in the Complaints team. 
Your role is to retrieve and analyze complaints based on user queries. 
Use the provided tools to filter, retrieve, and generate responses."""


messages = run_loop("Show me all complaints in region MC after 15th June 2024",system_prompts, tool_list)

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=4))
